# Earned Income Tax Credit

The Earned Income Tax Credit (EITC) is a refundable tax credit that phases in and out with earned income.
It is more generous for families with children, and it excludes filers with a certain amount of investment income.
Filers without children are only eligible if they are between 25 and 64 years of age.

In [14]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px


def make_eitc(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_household(name="household", members=members)
    sim.vary("employment_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            eitc=sim.calc("eitc")[0].round(),
            mtr=-sim.deriv("eitc", "employment_income", wrt_target="head"),
            adults=adults,
            children=children,
        )
    )


# Make a table of EITC amounts for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_eitc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    income_source="Income source",
    eitc="Earned income tax credit",
    mtr="EITC marginal tax rate",
    children="Children",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "eitc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="EITC for single filers by number of qualifying children, 2022",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

The EITC has marginal tax rates ranging from -45% to +21%, depending on the household structure and income.

In [13]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="EITC marginal tax rate for a single filer by number of qualifying children",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".0%",
)
fig.show()